In [1]:
# Calculate fission Spectra weighting 
import csv    # Library to read & process csv-formatted files
import sys    # Library to parse command-line arguments
import os
from math import sqrt, sinh, pi, exp
import numpy as np

rootdir=os.path.abspath(os.getcwd())

# Read in the spectra from file
fname = rootdir+"/E_fiss.csv"   # Fission spectra input file
e_bins=[]
fiss=[]
err=[]
try: 
    ifile = open(fname, 'r') 

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        e_bins.append(float(split_list[0].strip()))
        fiss.append(float(split_list[1].strip()))
        err.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)
    
np.asarray(fiss)
fiss=fiss/np.sum(fiss)

In [12]:
# Read in the bin structure from file
rootdir_235=os.path.abspath(os.path.join(os.path.abspath(os.path.join(rootdir, os.pardir)), os.pardir))+'/Validation/GEFY_5.3/235U/ChainYields'
rootdir_238=os.path.abspath(os.path.join(os.path.abspath(os.path.join(rootdir, os.pardir)), os.pardir))+'/Validation/GEFY_5.3/238U/ChainYields'
fname = rootdir+"/Bins.csv"   # Bin structure input file
bins=[]

try: 
    ifile = open(fname, 'r') 
    
    # Skip header line
    line=ifile.next()

    # Store the spectrum
    bins=[]
    for line in ifile:
        split_list=line.split(',')
        bins.append(float(split_list[2].strip()))

    # Close the file
    ifile.close()
except IOError as e:
    print "I/O error({0}): {1}".format(e.errno, e.strerror)
    print "File not found was: {0}".format(fname)

#print len(bins)
#print bins

49
[2.53e-08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0]


In [21]:
def Build_Weighted_FPs(bins,fw,datadir):
    one=0
    two=0
    y={}
    err={}
    w=[]
    fy=[]

    # Use weighted fission spectra to generate mass yield curves (sum yields?)
    for subdir, dirs, files in os.walk(datadir):
        for file in files:
            #print subdir, file
            print file
            
            # Determine Energy of File
            E=float(file.split('_')[0].strip())/1E6
            weight = 0
            one+=1
            
            # Determine weight of Energy bin
            for i in range(0,len(bins)):
                if abs(bins[i] - E) <= E*1E-4:
                    two+=1
                    weight=fw[i]
                    #print "Energy =", bins[i]
                    print "Fractional weight=",weight
            try: 
                ifile = open(subdir+'/'+file, 'r')
                reader = csv.reader(ifile)
                for line in reader:
                    # Build dictionaries
                    if not int(line[1]) in y.keys():  # this is the first time we've encountered this mass A
                        # let's initialize our dictionaries
                            y[int(line[1])] = float(line[2])*weight
                            err[int(line[1])] = (float(line[3])*weight)**2
                    else:  # we've encoutered this mass before, no need to initialize, just add to it
                        y[int(line[1])] += float(line[2])*weight
                        err[int(line[1])] += (float(line[3])*weight)**2

                    if int(line[1])==160:
                        w.append(weight)
                        fy.append(float(line[2]))
                # Close the file
                ifile.close()
            except IOError as e:
                print "I/O error({0}): {1}".format(e.errno, e.strerror)
                print "File not found was: {0}".format(fname)
    #print y
    #print y.keys()

    if one!=two or one!=len(bins):
        print "/n/nError: Not all of the files from {} were imported correctly./n/n".format(datadir)
        print one, two, len(bins)
        
    return y, err

In [35]:
#y={}
#err={}
#(y,err)=Build_Weighted_FPs(bins,fiss,rootdir_235)

y_235={}
err_235={}
f_235=1.0
(y_235,err_235)=Build_Weighted_FPs(bins,fiss*f_235,rootdir_235)

print "\n\n"
for A in y_235.keys():
    err_235[A]=sqrt(err_235[A])
    if A=='160':
        print y_235[A],err_235[A]
    
####Remove
y=y_235
err=err_235
####
    
#y_238={}
#err_238={}
#f_238=1-f_235
#(y_238,err_238)=Build_Weighted_FPs(bins,fiss*f_238,rootdir_238)

#for A in y_238.keys():
#    err_238[A]=sqrt(err_238[A])

#for A in y.keys():
#    try:
#        y[A]=y_235[A]+y_238[A]
#        err[A]=sqrt(err_235[A]**2+err_238[A]**2)
#    except KeyError as e:
#        print "Key error: {0} not found for both isotopes.".format(A)

1000000.0_u235_gefy_chain.csv
Fractional weight= 0.0282904083428
600000.0_u235_gefy_chain.csv
Fractional weight= 0.0410488121052
800000.0_u235_gefy_chain.csv
Fractional weight= 0.0333713098411
8000000.0_u235_gefy_chain.csv
Fractional weight= 0.00398364117477
2500000.0_u235_gefy_chain.csv
Fractional weight= 0.0488640144099
6000000.0_u235_gefy_chain.csv
Fractional weight= 0.00399623117848
12500000.0_u235_gefy_chain.csv
Fractional weight= 0.00429309126602
300000.0_u235_gefy_chain.csv
Fractional weight= 0.050312414837
4000000.0_u235_gefy_chain.csv
Fractional weight= 0.0132247038999
400000.0_u235_gefy_chain.csv
Fractional weight= 0.0515793152106
14000000.0_u235_gefy_chain.csv
Fractional weight= 0.117352034607
17000000.0_u235_gefy_chain.csv
Fractional weight= 2.76325081488e-06
8500000.0_u235_gefy_chain.csv
Fractional weight= 0.00379188111822
18500000.0_u235_gefy_chain.csv
Fractional weight= 5.5331016317e-07
9000000.0_u235_gefy_chain.csv
Fractional weight= 0.00360815106404
11500000.0_u235_gef

In [36]:
# Output the result
outfn="ETA_chain_fy.csv"

def writeout(outfn,outl):  # function to write our output in csv format
    ofile = open(outfn,"w")

    for out in outl:
        ofile.write(str(out[0])
                    + "," + str(out[1])
                    + "," + str(out[2]*100)
                    + "," + str(out[3]*100)
                    + "," + str(out[4])
                    + "\n")

    ofile.close()

outl=[]
for A in y.keys():
    outl.append((A,0,y[A],err[A]," "))
writeout(outfn,outl)